### Imports & Env Setup

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
import os
from dotenv import load_dotenv
load_dotenv()

import dspy
sys.path.append(os.path.abspath('../'))
from prompt_migrator.benchmarks import llama_mmlu_pro, leaderboard_mmlu_pro

/home/justinai/miniconda3/envs/prompt-migration/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


ModuleNotFoundError: No module named 'prompt_migrator'

In [2]:
pwd

'/home/justinai/Code/llama-cookbook/end-to-end-use-cases/prompt-migration/v2/notebooks'

### Configuration

In [ ]:
NUM_THREADS = 16

FEW_SHOTS = 5

# See https://docs.litellm.ai/docs/providers/vllm for details
TASK_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1' , # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)
PROMPT_MODEL = dspy.LM(
    "hosted_vllm/meta-llama/Llama-3.3-70B-Instruct",
    api_base = 'http://localhost:8000/v1', # or api_base ?
    # api_version: Optional[str] = None,
    # api_key: Optional[str] = None,
    # seed: Optional[int] = None,
    # max_tokens: Optional[int] = None,
    # timeout: Optional[Union[float, int]] = None,
)

dspy.configure(lm=TASK_MODEL)

# replace this with llama_mmlu_pro or whatever
benchmark = llama_mmlu_pro

# Without chain of thought:
# program = dspy.Predict(
#     benchmark.signature("")
# )

# With chain of thought:
program = dspy.ChainOfThought(
    benchmark.signature("") # put your initial system prompt here, or leave blank
)

evaluate = dspy.Evaluate(
    devset=[],
    metric=benchmark.metric,
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=True,
    return_all_scores=True,
    return_outputs=True,
)

### Load dataset

In [ ]:
trainset, valset, testset = benchmark.datasets(
    train_size=0.1,
    validation_size=0.2,
)

len(trainset), len(valset), len(testset)

### Optimize Subset + Evaluation

In [34]:
subset_size = 20
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="light",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 20

2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/15 17:44:49 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 20%|███████████████████████████████████████████████                                                                                                                                                                                            | 4/20 [00:20<01:23,  5.19s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


 40%|██████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                             | 8/20 [00:52<01:18,  6.56s/it]


...

2025/01/15 17:49:41 INFO dspy.evaluate.evaluate: Average Metric: 14 / 20 (70.0%)
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 70.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 75.0, 80.0, 75.0, 80.0, 70.0]
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 17:49:41 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 6 / 7 =====



Average Metric: 15.00 / 20 (75.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:24<00:00,  1.25s/it]

2025/01/15 17:50:06 INFO dspy.evaluate.evaluate: Average Metric: 15 / 20 (75.0%)
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 75.0, 80.0, 75.0, 80.0, 70.0, 75.0]
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 17:50:06 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



Average Metric: 15.00 / 20 (75.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.34s/it]

2025/01/15 17:50:33 INFO dspy.evaluate.evaluate: Average Metric: 15 / 20 (75.0%)
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 75.0 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4'].
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [25.0, 75.0, 80.0, 75.0, 80.0, 70.0, 75.0, 75.0]
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 80.0
2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/01/15 17:50:33 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 80.0!


In [35]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 To answer multiple-choice questions that require reasoning and analysis of the subject matter, follow these steps: 

1. Read the question carefully and identify the key concepts and information provided.
2. Analyze the options and determine which ones are plausible based on the information provided in the question.
3. Use a combination of natural language processing and knowledge retrieval to generate a step-by-step reasoning process to arrive at an answer.
4. Evaluate the reasoning process and select the most appropriate answer based on the analysis.
5. Provide a clear and concise explanation of the reasoning process used to arrive at the answer.

Given the fields `question`, `options`, produce the fields `reasoning`, `answer` by following the above steps and using a language model to generate a step-by-step explanation of how to arrive at the answer.


In [36]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
)

Average Metric: 13.00 / 20 (65.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.32s/it]

2025/01/15 17:52:03 INFO dspy.evaluate.evaluate: Average Metric: 13 / 20 (65.0%)


,question,options,example_answer,reasoning,pred_answer,metric
0,A certain map uses a scale of 1 inch equals 25 miles. How many mil...,"[A. 50, B. 150, C. 100, D. 25, E. 5, F. 200, G. 75, H. 125, I. 175...",H,"To find the number of miles represented by 5 inches on the map, we...",B,
1,When a ball at rest hangs by a single vertical string tension in t...,"['A. may be equal to mg depending on the speed of the ball', 'B. i...",D,"To solve this problem, we need to consider the forces acting on th...",D,✔️ [True]
2,"A 125-kW, 250-V, 1800-rev/min, cumulative compound d-c generator h...","['A. 90%, 200 hp', 'B. 87.5%, 195 hp', 'C. 84.7%, 175 hp', 'D. 88....",D,To find the efficiency and input horsepower requirements of the cu...,E,
3,The diameter of the objective lens of a telescope is 30 mm and the...,"[A. 80%, B. 70%, C. 100%, D. 30%, E. 40%, F. 90%, G. 25%, H. 60%, ...",B,To determine the fraction of the area of the objective lens that i...,B,✔️ [True]
4,"The temperature at the point $(x, y, z)$ in a substance with condu...","[A. $800\pi$, B. $1504\pi$, C. $1248\pi$, D. $1560\pi$, E. $960\pi...",C,To find the rate of heat flow inward across the cylindrical surfac...,C,✔️ [True]
5,Which of the following must be done when universal screening data ...,['A. The school must invest in more modern educational technology....,G,When universal screening data show that very few students are succ...,G,✔️ [True]
6,"Sulfurylchloride decomposes according to the equation, SO_2CI_2(g)...","['A. 1.49 × 10^-2, .0386', 'B. 7.45 × 10^-3, .0542', 'C. 2.98 × 10...",A,"To find the equilibrium constant K_p, we can use the equation: K_p...",A,✔️ [True]
7,"A number’s prime factors are 2, 5, 7, 13, and 31. Which of the fol...","[A. 10, B. 25, C. 6, D. 8, E. 15, F. 30, G. 20, H. 4]",A,"To find a factor that must be a factor of the number, we need to l...",A,✔️ [True]
8,How many ribs are there in the human body?,"[A. 42, B. 24, C. 18, D. 36, E. 40, F. 28, G. 22, H. 30, I. 20, J....",B,"The human body typically has 12 pairs of ribs, which are attached ...",B,✔️ [True]
9,To what extent have biological agents been weaponized?,['A. Military professionals believe that the majority of biologica...,C,The question asks about the extent to which biological agents have...,A,


## Medium Optimization

In [38]:
subset_size = 500
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="medium",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset[:subset_size],
    valset=valset[:subset_size],
    requires_permission_to_run=False,
)

2025/01/15 18:09:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 300

2025/01/15 18:09:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/15 18:09:23 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/15 18:09:23 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  1%|█▏                                                                                                                                                      | 4/500 [00:19<40:37,  4.91s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/19


  1%|█▏                                                                                                                                                      | 4/500 [00:17<36:04,  4.36s/it]


...

2025/01/15 19:59:49 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'The isomer of C_2H_5OH(l) is CH_3 - O - CH_3(g) . Given that, at 25°C, the heat of formation of C_2H_5OH(l) = - 66 kcal/mole, the heat of combustion of the isomeric CH_3 - O - CH_3(g) to CO_2(g) and H_2O(l) = - 348 kcal/mole, the heat of formation of H_2O(l) = - 68 kcal/mole and the heat of combustion of carbon to CO_2(g) = - 94 kcal/mole. Calculate a) \\DeltaH_298°K for the reaction and b) \\DeltaE_298°K for the reaction assuming \\DeltaH_298°K in part a) = - 10 kcal.', 'options': ['A. \\DeltaH = 22 kcal, \\DeltaE = - 10.6 kcal', 'B. \\DeltaH = 204 kcal, \\DeltaE = - 204 kcal', 'C. \\DeltaH = 204 kcal, \\DeltaE = 204 kcal', 'D. \\DeltaH = 348 kcal, \\DeltaE = - 188 kcal', 'E. \\DeltaH = 66 kcal, \\DeltaE = - 10 kcal', 'F. \\DeltaH = -66 kcal, \\DeltaE = -10 kcal', 'G. \\DeltaH = 66 kcal, \\DeltaE = 10 kcal', 'H. \\DeltaH = -22 kcal, \\DeltaE = 10.6 kcal', 'I. \\DeltaH = -348 kcal, \\DeltaE =

Average Metric: 217.00 / 298 (72.8%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 299/300 [05:03<00:02,  2.51s/it]

2025/01/15 20:02:20 ERROR dspy.utils.parallelizer: Error processing item Example({'question': "A 'fishbowl' of height 4r/3 is formed by removing the top third of a sphere of radius r=6. The fishbowl is fixed in sand so that its rim is parallel with the ground. A small marble of mass m rests at the bottom of the fishbowl. Assuming all surfaces are frictionless and ignoring air resistance, find the maximum initial velocity that could be given to the marble for it to land back in the fishbowl with g=9.8.", 'options': ['A. 20.5', 'B. 24.0', 'C. 16.4', 'D. 22.3', 'E. 18.25', 'F. 23.1', 'G. 21.7', 'H. 19.6', 'I. 17.0', 'J. 15.75'], 'answer': 'E'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 217.00 / 298 (72.8%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [05:29<00:00,  1.10s/it]

2025/01/15 20:02:20 INFO dspy.evaluate.evaluate: Average Metric: 217.0 / 300 (72.3%)
2025/01/15 20:02:20 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 72.33
2025/01/15 20:02:20 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [16.33, 70.33, 70.67, 72.33]
2025/01/15 20:02:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 72.33
2025/01/15 20:02:20 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/15 20:02:20 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/15 20:02:20 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 72.33!


In [39]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

BEST PROMPT:
 You are a skilled engineer and problem solver. Given the fields `question`, `options`, produce the fields `reasoning`, `answer` by carefully analyzing the problem, applying relevant formulas and principles, and selecting the most appropriate answer based on your calculations and logical deductions. Ensure your reasoning is step-by-step and clearly explains how you arrived at your answer.


In [41]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset[:subset_size],
    display_table=False,
)

Average Metric: 158.00 / 212 (74.5%):  42%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                 | 212/500 [03:52<05:18,  1.11s/it]

2025/01/16 10:35:33 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A microcomputer used for data acquisition and control is required to digitize and process four analog input signals and to output their average continually; i.e., in real time. The time for an external analog-to-digital converter (which is triggered by a CPU instruction) to digitize one input is 12 microseconds, and only one digitization occurs at a time. Five CPU instructions, including the triggering instruction, must be executed for each signal digitized. Ten CPU instructions are executed in order to average each set of four samples and output this value. The time to convert the output from digital to analog form is to be ignored. If it is assumed that suitable data buffering is employed, then the maximum average instruction execution time that allows the microcomputer to keep up with the input-output data rates, is', 'options': ['A. 3.0 microseconds', 'B. 2.6 microseconds', 'C. 2.4 micros

Average Metric: 174.00 / 234 (74.4%):  47%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                                      | 235/500 [04:17<04:51,  1.10s/it]

2025/01/16 10:35:58 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A 34 kg steel casting at a temperature of 427°C is quenched in136 kg of oil initially at 21°C. Assuming no heat losses and thesteel casting and oil to have constant specific heats of 0.5024and2.5121 kJ/kg-°K respectively, determine the - changein entropy for a system consisting of the oil and casting.', 'options': ['A. 11.99 kJ/°K', 'B. 13.99 kJ/°K', 'C. 6.99 kJ/°K', 'D. 10.99 kJ/°K', 'E. 5.99 kJ/°K', 'F. 9.99 kJ/°K', 'G. 4.99 kJ/°K', 'H. 8.99 kJ/°K', 'I. 12.99 kJ/°K', 'J. 7.99 kJ/°K'], 'answer': 'J'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 241.00 / 329 (73.3%):  66%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                      | 330/500 [06:11<03:36,  1.27s/it]

2025/01/16 10:37:50 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Consider the discrete memoryless channel $Y=XZ$ where $X$ and $Z$ are independent binary random variables that take on values 0 and 1. Let $P(Z=1)=0.5$. Find the capacity of this channel in bits.', 'options': ['A. 1.322', 'B. 0.500', 'C. 0.700', 'D. 0.100', 'E. 0.750', 'F. 1.000', 'G. 0.585', 'H. 0.322', 'I. 0.250', 'J. 0.811'], 'answer': 'H'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 257.00 / 350 (73.4%):  71%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                          | 353/500 [06:33<03:56,  1.61s/it]

2025/01/16 10:38:13 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'A neutron at rest decays (breaks up) to a proton and an electron. Energy is released in the decay and appears as kinetic energy of the proton and electron. The mass of a proton is 1836 times the mass of an electron. What fraction of the total energy released goes into the kinetic energy of the proton?', 'options': ['A. 0.1', 'B. 0.99', 'C. 0.5', 'D. 0.000544', 'E. 0.544', 'F. 0.00544', 'G. 0.1836', 'H. 0.0000918', 'I. 0.0544', 'J. 0.9180'], 'answer': 'D'}) (input_keys={'options', 'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys(['reasoning']). Set `provide_traceback=True` to see the stack trace.


Average Metric: 303.00 / 416 (72.8%):  84%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                        | 420/500 [07:47<01:18,  1.02it/s]

AttributeError: 'list' object has no attribute 'items'

## Heavy Optimization

In [ ]:
optimizer = dspy.MIPROv2(
    metric=benchmark.metric,
    auto="heavy",
    num_threads=NUM_THREADS,
    task_model=TASK_MODEL,
    prompt_model=PROMPT_MODEL,
    max_labeled_demos=FEW_SHOTS,
)

optimized_program = optimizer.compile(
    program,
    trainset=trainset,
    valset=valset,
)

In [ ]:
print("BEST PROMPT:\n", optimized_program.predict.signature.instructions)

In [ ]:
score, results, all_scores = evaluate(
    optimized_program,
    devset=testset,
    display_table=False,
)